In [ ]:
import math

# python3 -m pip install opencv-python
import cv2 # opencv
import numpy as np

from scipy.signal import convolve2d
from scipy.ndimage.filters import gaussian_filter

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
def imshow_with_colorbar(index, img):
    plt.figure(index)
    ax = plt.subplot(111)
    im = ax.imshow(img)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)

# 3.1 Blob detection

In [ ]:
img = cv2.imread('../images/sunflowers_small.jpg', cv2.IMREAD_GRAYSCALE)

plt.imshow(img, cmap='gray')

In [ ]:
kernel = [
    [ 1,  1,  1],
    [ 1, -8,  1],
    [ 1,  1,  1],
]
img_c = convolve2d(img, kernel, mode='valid')
imshow_with_colorbar(1, img_c)
imshow_with_colorbar(2, img_c[15:40,25:50])

# 3.2 Blob detection - Laplacian of Gaussian

In [ ]:
def generate_LoG_kernel(sz, sigma):
    def LoG(x, y, sigma):
        return ((x**2+y**2-2*sigma**2)/sigma**4)*math.exp(-1*(x**2+y**2)/(2*sigma**2))

    dsz = sz/2 - 1

    res = np.zeros((sz, sz), dtype=np.float)
    for y in range(sz):
        for x in range(sz):
            res[y,x] = LoG(x - dsz, y - dsz, sigma)
    return res

log_kernel = generate_LoG_kernel(31, 4)
imshow_with_colorbar(1, log_kernel)

log_kernel_2 = generate_LoG_kernel(31, 6)
imshow_with_colorbar(2, log_kernel_2)

In [ ]:
img = cv2.imread('../images/sunflowers_small.jpg', cv2.IMREAD_GRAYSCALE)

log_kernel_1 = generate_LoG_kernel(31, 1)
img_c_1 = convolve2d(img, log_kernel_1, mode='valid')
imshow_with_colorbar(1, img_c_1)

log_kernel_2 = generate_LoG_kernel(31, 2)
img_c_2 = convolve2d(img, log_kernel_2, mode='valid')
imshow_with_colorbar(2, img_c_2)

log_kernel_3 = generate_LoG_kernel(31, 3)
img_c_3 = convolve2d(img, log_kernel_3, mode='valid')
imshow_with_colorbar(3, img_c_3)

# 3.3 FAST features

In [ ]:
img = cv2.imread('../images/lenna.png', cv2.IMREAD_GRAYSCALE)
ff_detector = cv2.FastFeatureDetector_create(threshold=40)

kpts = ff_detector.detect(img)

img_out = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
# cv2.drawKeypoints(img, kpts, img_out)
for kpt in kpts:
    y = int(kpt.pt[1])
    x = int(kpt.pt[0])
    cv2.line(img_out, (x-5,y), (x+5,y), (255,0,0))
    cv2.line(img_out, (x,y-5), (x,y+5), (255,0,0))

plt.imshow(img_out)


# 3.4 Harris Corner Detector

In [ ]:
sigma = 3
test = np.zeros((21, 21), dtype=np.float)
test[11,11] = 1
test_out = np.zeros((21, 21), dtype=np.float)

test_out = gaussian_filter(test, sigma)

imshow_with_colorbar(1, test_out)

In [ ]:
def harris(im,sigma=3): # calculate Harris corner detector response function
    img = im.astype(np.float)

    # derivatives
    imgx = gaussian_filter(img, (sigma,sigma), (0,1))
    imgy = gaussian_filter(img, (sigma,sigma), (1,0))

    # components of the Harris matrix
    Wxx = gaussian_filter(imgx*imgx, sigma)
    Wxy = gaussian_filter(imgx*imgy, sigma)
    Wyy = gaussian_filter(imgy*imgy, sigma)

    # determinant and trace
    Wdet = Wxx*Wyy - Wxy**2
    Wtr = Wxx + Wyy

    return Wdet - 0.01 * Wtr * Wtr
    # return Wdet / Wtr

def harrisPoints(harrisim, min_dist=10, threshold=0.1):
    # find top corner candidates
    corner_threshold = harrisim.max() * threshold
    harrisim_t = (harrisim > corner_threshold) * 1
    
    # get coordinates of candidates
    coords = np.array(harrisim_t.nonzero()).T
    
    #get values of candidates
    candidate_values = [harrisim[c[0],c[1]] for c in coords]
    
    # sort candidates
    index = np.argsort(candidate_values)[::-1]
    
    # store allowed point locations in array
    allowed_locations = np.zeros(harrisim.shape)
    allowed_locations[min_dist:-min_dist,min_dist:-min_dist] = 1
    
    # select the best points using min_distance
    filtered_coords = []
    for i in index:
        if allowed_locations[coords[i,0],coords[i,1]] == 1:
            filtered_coords.append(coords[i])
            allowed_locations[(coords[i,0]-min_dist):(coords[i,0]+min_dist),
                        (coords[i,1]-min_dist):(coords[i,1]+min_dist)] = 0
    
    return filtered_coords

In [ ]:
img = cv2.imread('../images/lenna.png', cv2.IMREAD_GRAYSCALE)

img_harris = harris(img)
pts = harrisPoints(img_harris)

img_out = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

for pt in pts:
    y, x = pt[0], pt[1]
    cv2.line(img_out, (x-5,y), (x+5,y), (255,0,0))
    cv2.line(img_out, (x,y-5), (x,y+5), (255,0,0))

imshow_with_colorbar(1, img_harris)
imshow_with_colorbar(2, img_out)


# 3.4 Feature matching process

In [ ]:
img1 = cv2.imread('../images/curiosity_cut.jpg',cv2.IMREAD_GRAYSCALE)    # queryImage
img2 = cv2.imread('../images/curiosity_marker.jpg',cv2.IMREAD_GRAYSCALE) # trainImage

orb = cv2.ORB_create()

kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

matches = bf.match(des1,des2)

matches = sorted(matches, key = lambda x:x.distance)
img_match = cv2.drawMatches(img1,kp1,img2,kp2,matches[:10],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.imshow(img_match)